<a href="https://colab.research.google.com/github/milver/Experiments/blob/main/ChatGPT_Reducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reduces Size of ChatGPT Files

Mount Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import Libraries

In [ ]:
!pip install ijson
!pip install simplejson
import json
import os
import ijson
import simplejson as json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.6 MB/s eta 0:00:00


Give the application permission to mount the drive and create the folders

In [ ]:
# Create the Drive folders
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# This will create the folders if they don't exist.
folders =  ["ChatGPT_Conversations/", "ChatGPT_Conversations/ProcessedConversations/", "ChatGPT_Conversations/ReducedConversations/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)


Mounted at /content/drive


Conversation Extractor Functions

In [ ]:
# @title
# def process_chat_history(convo_path, output_convo_path):
#     """
#     Extracts conversations with speakers from a chat history file and saves them to a new JSON file.

#     Parameters:
#         convo_path (str): The path to the chat history file.
#         output_convo_path (str): The path to save the extracted conversations.

#     Returns:
#         None
#     """
#     with open(convo_path, 'r') as file:
#         chat_history = json.load(file)

#     extracted_conversations_with_speaker = []
#     for conversation in chat_history:
#     # Start from the current node and traverse backwards
#         current_node = conversation.get('current_node')
#         while current_node:
#             node = conversation.get('mapping', {}).get(current_node)
#             if node:
#                 message = node.get('message')
#                 if message:
#                     content = message.get('content', {}).get('parts', [])
#                     speaker = message.get('author', {}).get('role')
#                     is_user_system_message = message.get('metadata', {}).get('is_user_system_message', False)
#                     # Filter out system messages unless they are marked as user system messages
#                     if content and speaker and (speaker != "system" or is_user_system_message):
#                         # Change the speaker's name from "assistant" to "ChatGPT"
#                         if speaker == "assistant":
#                             speaker = "ChatGPT"
#                         # Handle the custom user info case for system messages
#                         elif speaker == "system" and is_user_system_message:
#                             speaker = "Custom user info"
#                         # Include both the content and the speaker in the extracted data
#                         for part in content:
#                             extracted_conversations_with_speaker.insert(0, {"speaker": speaker, "content": part})
#                 current_node = node.get('parent')

#     # Save the new structure to a new JSON file
#     with open(output_convo_path, 'w') as new_file:
#         json.dump(extracted_conversations_with_speaker, new_file, indent=4)

# def process_all_files(input_dir, output_dir):
#     """
#     Process all files in the given input directory and save the processed files
#     in the specified output directory.

#     Parameters:
#     - input_dir (str): The path to the directory containing the input files.
#     - output_dir (str): The path to the directory where the processed files will be saved.

#     Returns:
#     - None
#     """
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for filename in os.listdir(input_dir):
#         if filename.endswith('.json'):
#             input_convo_path = os.path.join(input_dir, filename)
#             output_convo_path = os.path.join(output_dir, f'processed_{filename}')
#             process_chat_history(input_convo_path, output_convo_path)

# Usage
# split_output_dir = 'split_json'  # Directory where split_json.py saves its output
# processed_output_dir = 'processed_json'  # New directory for processed files
# process_all_files(split_output_dir, processed_output_dir)

JSON Splitter

In [ ]:
# @title
# def split_json_file(input_file, output_dir, max_size_mb=10):
#     """
#     Split a JSON file into multiple smaller files based on a specified maximum size.

#     Args:
#         input_file (str): The path to the input JSON file.
#         output_dir (str): The directory where the split JSON files will be saved.
#         max_size_mb (int, optional): The maximum size in megabytes for each split file. Defaults to 10.

#     Raises:
#         FileNotFoundError: If the output directory does not exist.

#     Returns:
#         None
#     """
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     with open(input_file, 'rb') as file:
#         file_count = 1
#         current_size = 0
#         current_data = []

#         for item in ijson.items(file, 'item'):
#             json_str = json.dumps(item)
#             item_size = len(json_str.encode('utf-8'))

#             if current_size + item_size > max_size_mb * 1024 * 1024:
#                 output_file = os.path.join(output_dir, f'split_{file_count}.json')
#                 with open(output_file, 'w') as out_file:
#                     json.dump(current_data, out_file)

#                 file_count += 1
#                 current_size = 0
#                 current_data = []

#             current_data.append(item)
#             current_size += item_size

#         if current_data:
#             output_file = os.path.join(output_dir, f'split_{file_count}.json')
#             with open(output_file, 'w') as out_file:
#                 json.dump(current_data, out_file)

# Usage
# input_file = 'path\to\your\conversations.json'
# output_dir = 'split_json'
# split_json_file(input_file, output_dir)

## Upload any conversation files you want reduced in the "ChatGPT_Conversations" folder in your Google Drive

Process conversations.json

In [ ]:
from google.colab import drive
import os
import json
from datetime import datetime

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)  # This will prompt for authorization.

# Get the list of JSON files from the ChatGPT_Conversations folder
gpt_files = os.listdir("/content/drive/MyDrive/ChatGPT_Conversations")

# Function to convert timestamp to readable date
def convert_timestamp_to_date(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S') if timestamp else "Unknown Date"

# Loop through the JSON files and process them
for gpt_file in gpt_files:

    # Skip the file if it is not a JSON file
    if not gpt_file.endswith(".json"):
        continue

    # Load the conversation JSON file
    convo_path = "/content/drive/MyDrive/ChatGPT_Conversations/" + gpt_file

    with open(convo_path, 'r') as file:
        chat_history = json.load(file)

    # Process the chat history to include title and creation date as parent nodes
    extracted_conversations_with_metadata = []

    for conversation in chat_history:
        # Extract the title and creation date
        title = conversation.get("title", "Untitled Conversation")
        create_time = conversation.get("create_time")
        creation_date = convert_timestamp_to_date(create_time)

        # Extract the conversation flow
        extracted_conversations_with_speaker = []
        current_node = conversation.get('current_node')
        while current_node:
            node = conversation.get('mapping', {}).get(current_node)
            if node:
                message = node.get('message')
                if message:
                    content = message.get('content', {}).get('parts', [])
                    speaker = message.get('author', {}).get('role')
                    is_user_system_message = message.get('metadata', {}).get('is_user_system_message', False)
                    # Filter out system messages unless they are marked as user system messages
                    if content and speaker and (speaker != "system" or is_user_system_message):
                        # Change the speaker's name from "assistant" to "ChatGPT"
                        if speaker == "assistant":
                            speaker = "ChatGPT"
                        # Handle the custom user info case for system messages
                        elif speaker == "system" and is_user_system_message:
                            speaker = "Milver Valenzuela"
                        # Include both the content and the speaker in the extracted data
                        for part in content:
                            extracted_conversations_with_speaker.insert(0, {"speaker": speaker, "content": part})
                current_node = node.get('parent')

        # Add the metadata and conversation to the final structure
        extracted_conversations_with_metadata.append({
            "title": title,
            "creation_date": creation_date,
            "conversation": extracted_conversations_with_speaker
        })

    # Save the new structure to a new JSON file
    new_convo_path_with_metadata = "/content/drive/MyDrive/ChatGPT_Conversations/ReducedConversations/reduced_" + gpt_file
    with open(new_convo_path_with_metadata, 'w') as new_file:
        json.dump(extracted_conversations_with_metadata, new_file, indent=4)

    # Move the original file to the ProcessedConversations folder
    processed_path = "/content/drive/MyDrive/ChatGPT_Conversations/ProcessedConversations/" + gpt_file
    os.rename(convo_path, processed_path)

    # Print a message to indicate the progress
    print(f"Processed {gpt_file} and saved the reduced file with metadata in {new_convo_path_with_metadata}")


Mounted at /content/drive
Processed conversations_20241231.json and saved the reduced file with metadata in /content/drive/MyDrive/ChatGPT_Conversations/ReducedConversations/reduced_conversations_20241231.json


# Select and Export conversations into standalone json

In [ ]:
import gradio as gr
import json
import re
import os
from datetime import datetime
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Helper Functions
def load_json(file_path):
    """Load a JSON file from the specified file path with error handling."""
    try:
        print("Starting export process...")
        with open(file_path, 'r', encoding='utf-8') as file:
            global chat_data
            chat_data = json.load(file)
            conversation_list = [
                {
                    "title": conv.get("title", "Untitled"),
                    "creation_date": conv.get("creation_date", "Unknown Date")
                }
                for conv in chat_data
            ]
            return conversation_list
    except FileNotFoundError:
        return {"error": f"File not found. Check the path: {file_path}"}
    except json.JSONDecodeError as e:
        return {"error": f"Invalid JSON format. {e}"}
    except Exception as e:
        return {"error": f"An unexpected error occurred: {e}"}

def search_conversations(query):
    """Search conversations by title or content with error handling."""
    try:
        if not chat_data:
            return {"error": "Please load a JSON file first."}

        pattern = re.compile(query, re.IGNORECASE)
        results = []

        for idx, conv in enumerate(chat_data):
            if not isinstance(conv, dict):
                continue

            title = conv.get("title", "")
            if not isinstance(title, str):
                continue

            title_match = pattern.search(title)
            conversation = conv.get("conversation", [])
            snippets = []
            if isinstance(conversation, list):
                snippets = [
                    msg.get("content", "")
                    for msg in conversation
                    if isinstance(msg, dict) and isinstance(msg.get("content", ""), str) and pattern.search(msg.get("content", ""))
                ]

            if title_match:
                snippets.insert(0, f"Title Match: {title_match.group(0)}")

            if title_match or snippets:
                results.append({
                    "index": idx,
                    "title": title,
                    "creation_date": conv.get("creation_date", "Unknown Date"),
                    "snippets": snippets[:3]
                })

        return results

    except re.error as e:
        return {"error": f"Invalid regex pattern: {e}"}
    except Exception as e:
        return {"error": f"An unexpected error occurred during the search: {e}"}

def export_selected_conversations(selected_indices, export_folder):
    """Export selected conversations to JSON files with error handling."""
    try:
        if not chat_data:
            return {"error": "Please load a JSON file first."}
        if not selected_indices:
            return {"error": "No conversations selected for export."}

        if not export_folder.startswith("/"):
            export_folder = os.path.join("/content/drive/MyDrive", export_folder)
        if not os.path.exists(export_folder):
            os.makedirs(export_folder)

        exported_files = []
        for idx in selected_indices:
            if idx < len(chat_data):
                conv = chat_data[idx]
                title = conv.get("title", "Untitled")
                creation_date = conv.get("creation_date", "Unknown_Date")
                sanitized_title = re.sub(r'[\\/:*?"<>|]', '_', title)
                file_name = f"{sanitized_title}_{creation_date.replace(' ', '_')}.json"
                file_path = os.path.join(export_folder, file_name)

                with open(file_path, 'w', encoding='utf-8') as file:
                    json.dump(conv, file, indent=4)
                exported_files.append(file_path)

        return {"message": f"Exported {len(exported_files)} conversation(s) to {export_folder}"}
    except Exception as e:
        return {"error": f"An unexpected error occurred during export: {e}"}

# Global variable to store chat data
chat_data = []

# Gradio Interface
def handle_export(selected_rows, export_folder):
    debug_messages = []
    try:
        debug_messages.append("Export button clicked.")
        debug_messages.append(f"Selected rows: {selected_rows}")
        debug_messages.append(f"Export folder: {export_folder}")

        # Ensure selected_rows is a list of rows
        if hasattr(selected_rows, "to_dict"):
            selected_rows = selected_rows.to_dict(orient="records")

        if not selected_rows:
            debug_messages.append("No rows selected for export.")
            return {"error": "No rows selected for export.", "debug": debug_messages}

        if not export_folder or not isinstance(export_folder, str):
            debug_messages.append("Invalid export folder path.")
            return {"error": "Export folder path is not valid.", "debug": debug_messages}

        if not chat_data:
            debug_messages.append("Chat data is not loaded.")
            return {"error": "Please load a JSON file first.", "debug": debug_messages}

        debug_messages.append("Starting export process...")

        # Extract indices from the first column or 'Index' field
        selected_indices = []
        for row in selected_rows:
            if isinstance(row, dict) and "Index" in row:
                selected_indices.append(row["Index"])
            elif isinstance(row, (list, tuple)) and len(row) > 0:
                selected_indices.append(row[0])

        debug_messages.append(f"Selected indices: {selected_indices}")

        # Export the selected conversations
        results = export_selected_conversations(selected_indices, export_folder)
        debug_messages.append(f"Export result: {results}")

        if isinstance(results, dict) and "error" in results:
            debug_messages.append(f"Export failed with error: {results['error']}")
            return {"error": results["error"], "debug": debug_messages}

        debug_messages.append("Export process completed successfully.")
        return {"message": results["message"], "debug": debug_messages}
    except Exception as e:
        debug_messages.append(f"An unexpected error occurred: {e}")
        return {"error": f"An unexpected error occurred: {e}", "debug": debug_messages}


# Create Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("# ChatGPT Conversations Manager")
    gr.Markdown("Provide the full path to your JSON file in Google Drive, search conversations, and export selected results.")

    file_path_input = gr.Textbox(
        label="Full Path to JSON File",
        value="/content/drive/MyDrive/ChatGPT_Conversations/ReducedConversations/reduced_conversations_20241231.json"
    )
    run_app_button = gr.Button("Process JSON")
    response_output = gr.JSON(label="Response with Debug Messages")

    run_app_button.click(
        lambda file_path: load_json(file_path),
        inputs=[file_path_input],
        outputs=[response_output]
    )

    search_query_input = gr.Textbox(label="Search Query")
    export_folder_input = gr.Textbox(
        label="Export Folder",
        value="/content/drive/MyDrive/ChatGPT_Conversations/IndividualConversations"
    )
    search_results_table = gr.Dataframe(
        headers=["Index", "Title", "Creation Date", "Snippet"],
        value=[],
        interactive=True,
        datatype=["number", "str", "str", "str"]
    )
    export_button = gr.Button("Export Selected")

    def update_table(query):
        search_result = search_conversations(query)
        if isinstance(search_result, list):
            return [[res['index'], res['title'], res['creation_date'], ' | '.join(res['snippets'])] for res in search_result]
        return []

    search_query_input.change(
        lambda query: update_table(query),
        inputs=[search_query_input],
        outputs=[search_results_table]
    )

    export_button.click(
        handle_export,
        inputs=[search_results_table, export_folder_input],
        outputs=[response_output]
    )

# Launch Gradio app
interface.launch(share=True)


Mounted at /content/drive
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3539e812a6e0c1dedd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Chunks a conversation into manageable files

In [ ]:
# ===================================
# 1. Install necessary libraries
# ===================================
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.3 MB/s eta 0:00:00


In [39]:
# ===================================
# 2. Import libraries
# ===================================
import json
import os
import zipfile
from tiktoken import get_encoding

# ===================================
# 3. Define helper functions
# ===================================
def count_tokens(text, encoding_name="gpt2"):
    """
    Counts the number of tokens in a given text using the specified encoding.
    By default uses 'gpt2' encoding, which is a good approximation
    for GPT-3.5/GPT-4 token usage.
    """
    enc = get_encoding(encoding_name)
    return len(enc.encode(text))

def build_prompt_response_pairs(conversation):
    """
    Groups a conversation list of dicts into prompt–response pairs.
    Example structure of each conversation entry:
        {
            "speaker": "user" or "ChatGPT",
            "content": "..."
        }

    Logic:
    - For each 'user' message, gather subsequent 'ChatGPT' messages
      until the next 'user' appears.
    - Each pair is [ user_msg, chatgpt_msg1, chatgpt_msg2, ... ].
    - If there's a user without a ChatGPT response, that user remains as a standalone pair
      (though typically we want user + response).
    """
    pairs = []
    current_user = None
    current_chatgpt_responses = []

    for msg in conversation:
        speaker = msg.get("speaker", "").lower()
        if speaker == "user":
            # If there's a pending user + responses, finalize it first
            if current_user or current_chatgpt_responses:
                # Add the existing user-response pair
                # (only if we actually had a user)
                if current_user:
                    pair = [current_user] + current_chatgpt_responses
                    pairs.append(pair)

            # Start a new user
            current_user = msg
            current_chatgpt_responses = []

        elif speaker == "chatgpt":
            # If we have an ongoing user, add this ChatGPT message
            if current_user:
                current_chatgpt_responses.append(msg)
            else:
                # No user prompt to attach to; we could ignore or treat it as a leftover
                # For safety, you might store these as standalone "pairs" if desired.
                pass

    # After the loop, if there's a leftover user and responses, finalize it
    if current_user:
        pair = [current_user] + current_chatgpt_responses
        pairs.append(pair)

    return pairs

def pairs_to_text(pairs):
    """
    Given a list of pairs, where each pair is a list of dicts
    [ {speaker, content}, {speaker, content}, ... ],
    convert everything into a single concatenated string for token counting.
    """
    lines = []
    for pair in pairs:
        for msg in pair:
            speaker = msg.get("speaker", "")
            content = msg.get("content", "")
            lines.append(f"{speaker}: {content}")
    return "\n".join(lines)

# ===================================
# 4. Main chunking + zipping logic
# ===================================
def chunk_json_file_into_pairs(
    input_json_path,
    output_prefix="my_conversation_chunk",
    max_tokens=7000,          # ~7k is safer for GPT-4 8k context
    encoding_name="gpt2"
):
    """
    - Loads a JSON file with:
        {
          "title": ...,
          "creation_date": ...,
          "conversation": [
             { "speaker": ..., "content": ... },
             ...
          ]
        }
    - Groups the conversation into user->ChatGPT prompt–response pairs.
    - Chunks those pairs so that each output JSON stays under `max_tokens` tokens.
    - Each chunk is saved to disk and then all chunk files are zipped into:
        "chunked_<original-filename>.zip".
    """
    # 1. Load the input JSON
    with open(input_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    title = data.get("title", "")
    creation_date = data.get("creation_date", "")
    conversation = data.get("conversation", [])

    # 2. Build prompt–response pairs
    all_pairs = build_prompt_response_pairs(conversation)

    # We'll accumulate pairs into a chunk until it exceeds max_tokens
    chunk_filenames = []
    chunk_counter = 1

    current_chunk = []
    current_text = ""

    for pair in all_pairs:
        # Turn this pair into text for token count
        pair_text = pairs_to_text([pair])
        test_text = current_text + "\n" + pair_text
        test_token_count = count_tokens(test_text, encoding_name=encoding_name)

        if test_token_count <= max_tokens:
            # Add this pair to the current chunk
            current_chunk.append(pair)
            current_text = test_text
        else:
            # Save the current chunk, then start a new one
            if current_chunk:
                # Build chunk data in the same schema as the original,
                # but with conversation replaced by the chunk pairs flattened
                chunk_data = {
                    "title": title,
                    "creation_date": creation_date,
                    "conversation": [msg for pair_ in current_chunk for msg in pair_]
                }

                chunk_filename = f"{output_prefix}_{chunk_counter:03d}.json"
                with open(chunk_filename, "w", encoding="utf-8") as out_f:
                    json.dump(chunk_data, out_f, ensure_ascii=False, indent=4)

                print(f"Saved chunk #{chunk_counter}: {chunk_filename} "
                      f"(~{count_tokens(pairs_to_text(current_chunk), encoding_name)} tokens)")
                chunk_filenames.append(chunk_filename)
                chunk_counter += 1

            # Start a fresh chunk with the new pair
            current_chunk = [pair]
            current_text = pair_text

    # Save any leftover chunk
    if current_chunk:
        chunk_data = {
            "title": title,
            "creation_date": creation_date,
            "conversation": [msg for pair_ in current_chunk for msg in pair_]
        }
        chunk_filename = f"{output_prefix}_{chunk_counter:03d}.json"
        with open(chunk_filename, "w", encoding="utf-8") as out_f:
            json.dump(chunk_data, out_f, ensure_ascii=False, indent=4)

        print(f"Saved final chunk #{chunk_counter}: {chunk_filename} "
              f"(~{count_tokens(pairs_to_text(current_chunk), encoding_name)} tokens)")
        chunk_filenames.append(chunk_filename)

    # 3. Zip all the chunks
    base_name = os.path.splitext(os.path.basename(input_json_path))[0]
    zip_filename = f"chunked_{base_name}.zip"
    with zipfile.ZipFile(zip_filename, 'w') as zf:
        for cf in chunk_filenames:
            zf.write(cf)

    print(f"\nAll chunks have been zipped into: {zip_filename}")
    print("Chunk files included:")
    for cf in chunk_filenames:
        print(f"  - {cf}")

# ===================================
# 5. Example usage in Colab
# ===================================

# Replace 'transcripts.json' with the actual file you have in your Colab environment.
input_file_path = "/content/drive/MyDrive/ChatGPT_Conversations/IndividualConversations/LinkedIn Networking and Recruiting_2024-08-15_17:13:45.json"

# This function will:
#  - read the JSON
#  - group conversation into pairs
#  - chunk them up to ~7000 tokens each (safe for GPT-4 8K limit)
#  - save each chunk as a JSON file
#  - zip them all into chunked_transcripts.zip
chunk_json_file_into_pairs(
    input_json_path=input_file_path,
    output_prefix="my_conversation_chunk",
    max_tokens=7000,   # ~7K tokens for safety
    encoding_name="gpt2"
)


Saved chunk #1: my_conversation_chunk_001.json (~6586 tokens)
Saved chunk #2: my_conversation_chunk_002.json (~6388 tokens)
Saved chunk #3: my_conversation_chunk_003.json (~4240 tokens)
Saved chunk #4: my_conversation_chunk_004.json (~5337 tokens)
Saved chunk #5: my_conversation_chunk_005.json (~6188 tokens)
Saved chunk #6: my_conversation_chunk_006.json (~6757 tokens)
Saved final chunk #7: my_conversation_chunk_007.json (~3444 tokens)

All chunks have been zipped into: chunked_LinkedIn Networking and Recruiting_2024-08-15_17:13:45.zip
Chunk files included:
  - my_conversation_chunk_001.json
  - my_conversation_chunk_002.json
  - my_conversation_chunk_003.json
  - my_conversation_chunk_004.json
  - my_conversation_chunk_005.json
  - my_conversation_chunk_006.json
  - my_conversation_chunk_007.json
